In [1]:
# import package
import asl

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# Example videos

In [2]:
# path to video
path = 'Examples/IMG_0976.MOV'

# load video from file
V = asl.video_file(path)
print(V)

2023-11-09 09:31:48.100833: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


 path: Examples/IMG_0976.MOV
 Resolution: (1080, 1920)
 Number of frames: 446
 Percentage of frames with landmarks: 0.968609865470852


In [3]:
asl.write_video(V)

Video saved


'IMG_0976_annotated.avi'

# Video

In [4]:
asl.show_video(V)

interactive(children=(IntSlider(value=0, description='i', layout=Layout(width='1000px'), max=445), Output()), …

<function asl.converter.show_annotated_video.<locals>.show_frame(i)>